Import necessary libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import os 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
import re
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Load the input dataset

In [2]:
Input_urls=pd.read_excel('Input.xlsx')
Input_urls.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


_Get the HTML_

In [ ]:
# print(Input_urls.columns)
for urls in range(len(Input_urls)):
    r=requests.get(Input_urls['URL'][urls])
    htmlcontent=r.content
    soup=BeautifulSoup(htmlcontent,'html.parser')

    # Extract text from all <p> tags
    if(soup.article!=None):
       Title= soup.article.find('h1')
       title=Title.get_text()
       paragraphs = soup.article.find_all('p')
       text_content = "\n".join([p.get_text() for p in paragraphs])
    
    # Save text content to a file with URL name as the filename
    filename = "\Input_scraped\\"+f"{urls+1}_{Input_urls['URL'][urls].replace('/', '_').replace(':', '_')}.txt"
    
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"{title}\n\n")
        file.write(text_content)

    # print(f"Text content from {Input_urls['URL'][urls]} saved to {filename}")

_Directories_used_

In [34]:
text_dir ='\Input_scraped'
stopwords_dir = "\StopWords"
master_dir = "\MasterDictionary"


Loop over all files in stopwords directory

In [25]:
stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))


Loop over all files in text_dir

In [32]:
docs=[]
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir,text_file),'r',encoding='utf-8') as f:
        text=f.read()
        #tokenize the given text file
        words = word_tokenize(text)
        # remove the stop words from the tokens
        filtered_text = [word for word in words if word.lower() not in stop_words]
        # add each filtered tokens of each file into a list
        docs.append(filtered_text)
        

In [ ]:
print(docs)

Storing positive and negative word from MasterDictionary   

In [36]:
pos=set()
neg=set()

for files in os.listdir(master_dir):
    if(files=='positive-words.txt'):
        with open(os.path.join(master_dir,files),'r') as f:
            pos.update(f.read().splitlines())
    elif(files=='negative-words.txt'):
        with open(os.path.join(master_dir,files),'r') as f:
            neg.update(f.read().splitlines())   
            



Now collect the positive  and negative words from each file

In [50]:
positive_word=[]
negative_word=[]
positive_score=[]
negative_score=[]
polarity_score=[]
subjectivity_score=[]

for i in range(len(docs)): 
    positive_word.append([word for word in docs[i] if word.lower() in pos])
    negative_word.append([word for word in docs[i] if word.lower() in neg])
    positive_score.append(len(positive_word[i]))
    negative_score.append(len(negative_word[i]))
    polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
    subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

Determinig output variables 

In [65]:
# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words 
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

stopwords = set(stopwords.words('english'))
def measure(file):
  with open(os.path.join(text_dir, file),'r',encoding='utf-8') as f:
    text = f.read()
# remove punctuations 
    text = re.sub(r'[^\w\s.]','',text)
# split the given text file into sentences
    sentences = text.split('.')
# total number of sentences in a file
    num_sentences = len(sentences)
# total words in the file
    words = [word  for word in text.split() if word.lower() not in stopwords ]
    num_words = len(words)
 
# complex words having syllable count is greater than 2
# Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word


    avg_sentence_len = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words  =  len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count

Iterate through each file

In [66]:
for file in os.listdir(text_dir):
  x,y,z,a,b = measure(file)
  avg_sentence_length.append(x)
  Percentage_of_Complex_words.append(y)
  Fog_Index.append(z)
  complex_word_count.append(a)
  avg_syllable_word_count.append(b)

In [69]:
# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by 
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r',encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  x, y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)


# To calculate Personal Pronouns mentioned in the text, we use regex to find 
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r',encoding='utf-8') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(text_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [74]:
Output_urls=pd.read_excel('Output Data Structure.xlsx')

variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
  Output_urls.iloc[:,i+2] = var

#now save the dataframe to the disk
Output_urls.to_excel('Output_Data.xlsx')
    
